作业：
1.运行课堂代码，对比两种迁移学习方法在本数据集中的效果，可以尝试增加epoch；
2.尝试更多的预训练网络和不同的正则化方法，可以finetuning一层或多层，在cifar10的测试集中尽可能获得高的准确率
选做：
1.实现L1正则化，并与L2正则化对比效果，框架不限。

1.
resnet18模型不冻结梯度，epoch=30,对蜜蜂蚂蚁图片进行分类:
耗时4m46s，最高的验证集准确率是0.947712
resnet18模型冻结所有梯度除最后一个全连接层，epoch=30,对蜜蜂蚂蚁图片进行分类:
耗时3m32s，最高的验证集准确率是0.947712   
从验证集准确率结果看，两个模型一致，分析一下可能有两个主要原因：resnet18预训练模型比较有效，
蜜蜂蚂蚁二分类问题比较简单。

2.用resnet18、alexnet 预训练网络，对cifar10数据进行分类，
resnet18:
除最后一个全连接层,冻结所有梯度。
epoch=10，batch_size=100,resnet18耗时 17m 40s，最后一次epoch:train Loss: 1.0113 Acc: 0.6475，val Loss: 0.7197 Acc: 0.7572，test Loss: 0.7833 Acc: 0.7402。
epoch=10，batch_size=200,优化器加入L2正则化，weigt_decay=0.1,resnet18耗时 17m 38s，最后一次epoch:train Loss: 1.1105 Acc: 0.6370,val Loss: 0.8621 Acc: 0.7400,test Loss: 0.9084 Acc: 0.7317.
不冻结梯度：epoch=10，batch_size=100，不加入L2正则化，resnet18耗时36m 32s，最后一次epoch:train Loss: 0.9171 Acc: 0.8174，val Loss: 0.6759 Acc: 0.8976，test Loss: 0.6858 Acc: 0.8957。

alexnet:
除最后一个全连接层,冻结所有梯度。
epoch=10，batch_size=100,去掉lr递减,alexnet耗时13m31s,最后一次epoch:train Loss: 0.9566 Acc: 0.6615,val Loss: 0.6143 Acc: 0.7882,test Loss: 0.6159 Acc: 0.7863.
epoch=10，batch_size=100,去掉lr递减,优化器加入L2正则化，alexnet耗时13m24s,最后一次epoch:train Loss: 0.9929 Acc: 0.6452,val Loss: 0.6513 Acc: 0.7656,test Loss: 0.6827 Acc: 0.7639.
不冻结梯度：epoch=10，batch_size=100，去掉lr递减,alexnet耗时20m12s,最后一次epoch:train Loss: 0.6418 Acc: 0.7768,val Loss: 0.3774 Acc: 0.8732
,test Loss: 0.3728 Acc: 0.8722。


结论：
1.数据增广对于提高准确率有很大帮助，没做transforms.RandomResizedCrop(128),  transforms.Resize(128),transforms.CenterCrop(128)等之前，迁移学习（除全连接层之外全冻结）准确率一直在50%以下。
2.不冻结的准确率明显更高
3.L2正则化与lr递减相比，貌似没什么效果。


In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import warnings
warnings.filterwarnings("ignore")

In [2]:
data_transforms={
    'train':transforms.Compose([
        transforms.RandomResizedCrop(128),      
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    "val":transforms.Compose([
        transforms.Resize(128),
        transforms.CenterCrop(128),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    "test":transforms.Compose([
        transforms.Resize(128),
        transforms.CenterCrop(128),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.48216, 0.44653),(0.24703, 0.24349, 0.26159))
    ])
}


trainset = torchvision.datasets.CIFAR10(root='./data', train=True, 
                                        download=True, transform=data_transforms['train'])
valset = torchvision.datasets.CIFAR10(root='./data', train=True, 
                                       download=True, transform=data_transforms['val'])
testset = torchvision.datasets.CIFAR10(root='./data', train=False, 
                                       download=True, transform=data_transforms['test'])
        
indices = np.arange(50000)
np.random.shuffle(indices)       
train_loader = torch.utils.data.DataLoader(trainset, batch_size=100, 
                                          shuffle=False, sampler=torch.utils.data.SubsetRandomSampler(indices[:45000]))
val_loader = torch.utils.data.DataLoader(valset, batch_size=100, 
                                         shuffle=False, sampler=torch.utils.data.SubsetRandomSampler(indices[45000:50000]))
test_loader = torch.utils.data.DataLoader(testset, batch_size=100, 
                                         shuffle=False, num_workers=4)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:
inputs, classes = next(iter(train_loader))
inputs.size()
len(trainset)
len(valset)

torch.Size([100, 3, 128, 128])

50000

50000

In [3]:
dataloaders={'train':train_loader,'val':val_loader,"test":test_loader}
dataset_sizes = {'train':45000,'val':5000,"test":10000}

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val',"test"]:
            if phase == 'train':
                #scheduler.step()
                model.train()  # Set model to training mode
                
            else:
                model.eval()   # Set model to evaluate mode
    
                
            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
           
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if (phase == 'val'or  phase == "test") and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val/test Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [5]:
model_ft = models.alexnet(pretrained = True)
for param in model_ft.parameters():
    param.requires_grad = False
num_ftrs = model_ft.classifier[6].in_features
model_ft.classifier[6] = nn.Linear(num_ftrs, 10)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(),lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model_ft
for param in  model_ft.parameters():
    print(param.requires_grad)

In [6]:
train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=10)

Epoch 0/9
----------
train Loss: 1.2285 Acc: 0.5767
val Loss: 0.6080 Acc: 0.7974


RuntimeError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 2.00 GiB total capacity; 1022.10 MiB already allocated; 313.96 MiB free; 1.01 GiB reserved in total by PyTorch)